# YAML Model

In [114]:
import ZODB
from Products.Five.browser.tests.pages import SimpleView
from Testing.makerequest import makerequest 							# makerequest(context)
from Testing.ZopeTestCase.testZODBCompat import make_request_response 	# make_request_response()[1]

from Products.zms import standard

# Basic
import os
import re
import shutil
import tempfile
import urllib
import json
import requests
import yaml
import sys

## Step-2: Open ZODB database

In [116]:
# Create a ZODB connection to an existing ZODB database file
# Important: Fit working directory to your Zope instance

try:
	wd = '/home/zope/instance/zms5_dev/var'
	db = ZODB.DB(os.path.join(wd, 'Data.fs'))
	conn = db.open()
	root = conn.root
	###{'Application': <Application at >}
except:
	db.close()
	print('Error: Database connection had to be closed before reopened.')

## Step-3: Get ZODB Context

In [117]:
# ZMS-Node /myzmsx/content
# ##############################################
context = root.Application.myzmsx.content 
# ##############################################

# Add REQUEST to zmscontext object
context = makerequest(context)
# Add REQUEST vars
context.REQUEST.environ.setdefault('SERVER_NAME','localhost')
context.REQUEST.environ.setdefault('SERVER_PORT', '8092')
context.REQUEST['URL' ]= 'http://localhost:8092'
context.REQUEST.set('lang','ger')
context.REQUEST.set('path_to_handle','')
# Add RESPONSE
context.REQUEST.set('RESPONSE', make_request_response()[1])

zmscontext = context

In [122]:
zmscontext.getMetaobjManager().getMetaobj('ZMSFlexbox')['attrs']

[{'id': 'icon_clazz',
  'name': 'Icon-Class (CSS)',
  'mandatory': 0,
  'multilang': 0,
  'repetitive': 0,
  'type': 'constant',
  'keys': [],
  'custom': 'icon-columns fas fa-columns',
  'default': ''},
 {'id': 'e',
  'name': 'Items',
  'mandatory': 0,
  'multilang': 0,
  'repetitive': 1,
  'type': '*',
  'keys': ['type(ZMSObject)'],
  'custom': '',
  'default': ''},
 {'id': 'f_css_defaults',
  'name': 'CSS Default',
  'mandatory': 0,
  'multilang': 0,
  'repetitive': 0,
  'type': 'constant',
  'keys': [],
  'custom': '@media only screen and (min-width: 576px) {\r\n\tbody:not(.zmi) .ZMSFlexbox:not(.contentEditable) {\r\n\t\tdisplay: flex;\r\n\t\tflex-direction: row;\r\n\t\tjustify-content: space-between;\r\n\t}\r\n\tbody:not(.zmi) .ZMSFlexbox:not(.contentEditable) .ZMSFlexboxItem {\r\n\t\tflex-direction: column;\r\n\t\tbackground: rgba(0,0,0,.05);\r\n\t\tpadding: 1rem;\r\n\t\tmargin:1rem 0;\r\n\t}\r\n\tbody:not(.zmi) .ZMSFlexbox:not(.contentEditable) .ZMSFlexboxItem > * {\r\n\t\tmargi

In [125]:
data = zmscontext.getMetaobjManager().getMetaobj('ZMSFlexbox')['attrs'][2]
data

{'id': 'f_css_defaults',
 'name': 'CSS Default',
 'mandatory': 0,
 'multilang': 0,
 'repetitive': 0,
 'type': 'constant',
 'keys': [],
 'custom': '@media only screen and (min-width: 576px) {\r\n\tbody:not(.zmi) .ZMSFlexbox:not(.contentEditable) {\r\n\t\tdisplay: flex;\r\n\t\tflex-direction: row;\r\n\t\tjustify-content: space-between;\r\n\t}\r\n\tbody:not(.zmi) .ZMSFlexbox:not(.contentEditable) .ZMSFlexboxItem {\r\n\t\tflex-direction: column;\r\n\t\tbackground: rgba(0,0,0,.05);\r\n\t\tpadding: 1rem;\r\n\t\tmargin:1rem 0;\r\n\t}\r\n\tbody:not(.zmi) .ZMSFlexbox:not(.contentEditable) .ZMSFlexboxItem > * {\r\n\t\tmargin:0 !important;\r\n\t\tpadding:0 !important;\r\n\t}\r\n\tbody:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem {\r\n\t\tflex-basis:calc((100% / var(--data-itemcount)) - 3rem);\r\n\t}\r\n\tbody:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem .ZMSGraphic .graphic,\r\n\tbody:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem .ZMSGraphic .graphic img {\r\n\t\twidth:100% !important;\r\n\t\tmax-width:100% !important

In [126]:
out_zms = data.copy()
out_zms['custom'] = standard.str_yaml(out_zms['custom'])
print(out_zms)

{'id': 'f_css_defaults', 'name': 'CSS Default', 'mandatory': 0, 'multilang': 0, 'repetitive': 0, 'type': 'constant', 'keys': [], 'custom': '|\n@media only screen and (min-width: 576px) {\n\tbody:not(.zmi) .ZMSFlexbox:not(.contentEditable) {\n\t\tdisplay: flex;\n\t\tflex-direction: row;\n\t\tjustify-content: space-between;\n\t}\n\tbody:not(.zmi) .ZMSFlexbox:not(.contentEditable) .ZMSFlexboxItem {\n\t\tflex-direction: column;\n\t\tbackground: rgba(0,0,0,.05);\n\t\tpadding: 1rem;\n\t\tmargin:1rem 0;\n\t}\n\tbody:not(.zmi) .ZMSFlexbox:not(.contentEditable) .ZMSFlexboxItem > * {\n\t\tmargin:0 !important;\n\t\tpadding:0 !important;\n\t}\n\tbody:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem {\n\t\tflex-basis:calc((100% / var(--data-itemcount)) - 3rem);\n\t}\n\tbody:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem .ZMSGraphic .graphic,\n\tbody:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem .ZMSGraphic .graphic img {\n\t\twidth:100% !important;\n\t\tmax-width:100% !important;\n\t}\n\tbody:not(.zmi) .ZMSFlexbox .ZMSFlexbox

In [127]:
from ruamel.yaml import YAML
from ruamel.yaml.scalarstring import LiteralScalarString

yaml = YAML()
yaml.preserve_quotes = True

out_yaml = data.copy()
out_yaml['custom'] = LiteralScalarString(out_yaml['custom'])
yaml.dump(out_yaml, sys.stdout)


id: f_css_defaults
name: CSS Default
mandatory: 0
multilang: 0
repetitive: 0
type: constant
keys: []
custom: |-
  @media only screen and (min-width: 576px) {
  	body:not(.zmi) .ZMSFlexbox:not(.contentEditable) {
  		display: flex;
  		flex-direction: row;
  		justify-content: space-between;
  	}
  	body:not(.zmi) .ZMSFlexbox:not(.contentEditable) .ZMSFlexboxItem {
  		flex-direction: column;
  		background: rgba(0,0,0,.05);
  		padding: 1rem;
  		margin:1rem 0;
  	}
  	body:not(.zmi) .ZMSFlexbox:not(.contentEditable) .ZMSFlexboxItem > * {
  		margin:0 !important;
  		padding:0 !important;
  	}
  	body:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem {
  		flex-basis:calc((100% / var(--data-itemcount)) - 3rem);
  	}
  	body:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem .ZMSGraphic .graphic,
  	body:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem .ZMSGraphic .graphic img {
  		width:100% !important;
  		max-width:100% !important;
  	}
  	body:not(.zmi) .ZMSFlexbox .ZMSFlexboxItem .ZMSGraphic .text {
  		padding-top: